# Neural Network Applications for Applications in Stock markets - Predictions and Findings

In [2]:
import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
#import pandas_datareader as pdr
import pandas as pd
import csv
import warnings
from keras import layers
from keras import models
from numpy import array
from numpy import argmax
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


N = 20

Using TensorFlow backend.
/home/jesse/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jesse/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jesse/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jesse/.local/lib/python3.6/site-packages/tensorflow/python/fra

In [ ]:
%matplotlib inline

In [ ]:
#hide all warnings
#warnings.filterwarnings('ignore')

#show all warnings just once
warnings.filterwarnings(action='once')

In [ ]:
from helpers import delete_nans, get_returns, plot_example_returns, tickers, get_data_subsets#, data_loader

# Download Data

In [ ]:
begin = "2007-01-01"
finish = "2017-01-01"

#data_loader(begin, finish, 'returns.csv')

In [ ]:
returns = get_returns('returns.csv', N)

## Transform Data into different formats

In [ ]:
delete_nans('returns.csv', 'returns.csv')
plot_example_returns('returns.csv', N)

## Identify Branches, use One-Hot Encoding

In [3]:
pd.read_csv('snp500info.csv', index_col=0).head()
branches_ = pd.read_csv('snp500info.csv', index_col=0)
branches = branches_.drop(columns = ['Security', 'GICS Sub Industry', 'start at yahoo'])
branches.head()


,GICS Sector
Ticker symbol,
MMM,Industrials
ABT,Health Care
ABBV,Health Care
ACN,Information Technology
ATVI,Information Technology


In [ ]:
data = branches.iloc[:,0]
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(data)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)

# Network Architectures 

## CNN

In [ ]:
def cnn(x_train, y_train, x_test, y_test, inp_shape):
    verbose, epochs, batch_size = 1, 40, 50 
    """
    verbose: 0-kein output, 1-ladebalken, 2-epochenzahlen printen
    batch_size: Nicht definieren (https://stackoverflow.com/questions/44747343/keras-input-explanation-input-shape-units-batch-size-dim-etc)
    epochs: Anzahl Iterationen durch das Trainingsset
    """
    
    N = inp_shape[1]
    #init
    model = models.Sequential()
    model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu',
                     input_shape=inp_shape))
    model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu'))
    #Conv1D => 1D Convolution (Schaut nach Mustern)
    #mit filters = Anzahl Weight Functions, kernel_size = Anzahl simultan betrachteter Felder, 
    #relu = 0 für value<0 sonst linear
    
    model.add(layers.Dropout(0.3))
    #Dropout sets randomly chosen values to 0 to prevent overfitting
    
    model.add(layers.MaxPooling1D(pool_size=2))
    #MaxPooling halbiert array Größe und nimmt größte Werte der Feature-Gewichtungen 
    
    model.add(layers.Flatten())
    #Flatten reduziert dimensionen eines arrays auf niedrigst mögliche Dimension (1xdim) (überschreibt Nullen))
    
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(N, activation='linear'))
    #Klassisches NN hinter Convolutional Layer geschaltet, lernt also im Feature Raum, durch Convolutional Net vorgebenen
    
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse', 'mae']) 
    #mean_squared_error (mse) and mean_absolute_error (mae)
    
    #fit network
    model.fit(x_train,y_train, epochs=epochs, #batch_size=batch_size, 
              verbose=verbose)
    #evaluate model
    
    #Print error values for classification of goodness
    mse,mse2,mae = model.evaluate(x_test,y_test, batch_size=batch_size, verbose=verbose)
    print(mse)
    print(mse2)
    print(mae)
    return model
    

## Fully Connected NN

In [ ]:
def fully(x_train,y_train,x_test,y_test,inp_shape):
    verbose, epochs, batch_size = 1, 40, 50 
    """
    verbose: 0-kein output, 1-ladebalken, 2-epochenzahlen printen
    batch_size: Nicht definieren (https://stackoverflow.com/questions/44747343/keras-input-explanation-input-shape-units-batch-size-dim-etc)
    epochs: Anzahl Iterationen durch das Trainingsset
    """
    
    N = inp_shape[1]
    #init
    model = models.Sequential()
    model.add(layers.Dense(100, activation='tanh'))
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dropout(.5))
    model.add(layers.Dense(100, activation='tanh'))
    model.add(layers.Flatten())
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(N, activation='linear'))
    
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse', 'mae']) 
    #mean_squared_error (mse) and mean_absolute_error (mae)
    
    #fit network
    model.fit(x_train,y_train, epochs=epochs, #batch_size=batch_size, 
              verbose=verbose)
    #evaluate model
    
    #Print error values for classification of goodness
    mse,mse2,mae = model.evaluate(x_test,y_test, batch_size=batch_size, verbose=verbose)
    print(mse)
    print(mse2)
    print(mae)
    return model
    

## RNN

In [ ]:
def rnn(x_train, y_train, x_test, y_test, inp_shape):
    verbose, epochs, batch_size = 1, 40, 50 
    """
    verbose: 0-kein output, 1-ladebalken, 2-epochenzahlen printen
    batch_size: Nicht definieren (https://stackoverflow.com/questions/44747343/keras-input-explanation-input-shape-units-batch-size-dim-etc)
    epochs: Anzahl Iterationen durch das Trainingsset
    """
    
    N = inp_shape[1]
    #init
    model = models.Sequential()
    model.add(layers.GRU(42, input_shape = inp_shape, return_sequences = True))
    model.add(layers.Dropout(0.2))
    model.add(layers.GRU(42, return_sequences = False))
    
    #Dropout sets randomly chosen values to 0 to prevent overfitting
    
    #model.add(layers.MaxPooling1D(pool_size=2))
    #MaxPooling halbiert array Größe und nimmt größte Werte der Feature-Gewichtungen 
    
    #model.add(layers.Flatten())
    #Flatten reduziert dimensionen eines arrays auf niedrigst mögliche Dimension (1xdim) (überschreibt Nullen))
    
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(N, activation='linear'))
    #Klassisches NN hinter Convolutional Layer geschaltet, lernt also im Feature Raum, durch Convolutional Net vorgebenen
    
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse', 'mae']) 
    #mean_squared_error (mse) and mean_absolute_error (mae)
    
    #fit network
    model.fit(x_train,y_train, epochs=epochs, #batch_size=batch_size, 
              verbose=verbose)
    #evaluate model
    
    #Print error values for classification of goodness
    mse,mse2,mae = model.evaluate(x_test,y_test, batch_size=batch_size, verbose=verbose)
    print(mse)
    print(mse2)
    print(mae)
    return model


## LSTM

In [ ]:
def lstm(x_train, y_train, x_test, y_test, inp_shape):
    verbose, epochs, batch_size = 1, 40, 50 
    """
    verbose: 0-kein output, 1-ladebalken, 2-epochenzahlen printen
    batch_size: Nicht definieren (https://stackoverflow.com/questions/44747343/keras-input-explanation-input-shape-units-batch-size-dim-etc)
    epochs: Anzahl Iterationen durch das Trainingsset
    """
    
    N = inp_shape[1]
    D = inp_shape[0]
    
    hidden = int(2/3*(N*D))
    
    #init
    model = models.Sequential()
    model.add(layers.LSTM(hidden, input_shape = inp_shape, return_sequences = True))
    model.add(layers.Dropout(0.2))
    model.add(layers.LSTM(hidden, return_sequences = False))
    
    #Dropout sets randomly chosen values to 0 to prevent overfitting
    
    #model.add(layers.MaxPooling1D(pool_size=2))
    #MaxPooling halbiert array Größe und nimmt größte Werte der Feature-Gewichtungen 
    
    #model.add(layers.Flatten())
    #Flatten reduziert dimensionen eines arrays auf niedrigst mögliche Dimension (1xdim) (überschreibt Nullen))
    
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(N, activation='linear'))
    #Klassisches NN hinter Convolutional Layer geschaltet, lernt also im Feature Raum, durch Convolutional Net vorgebenen
    
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse', 'mae']) 
    #mean_squared_error (mse) and mean_absolute_error (mae)
        #fit network
    model.fit(x_train,y_train, epochs=epochs, #batch_size=batch_size, 
              verbose=verbose)
    #evaluate model
    
    #Print error values for classification of goodness
    mse,mse2,mae = model.evaluate(x_test,y_test, batch_size=batch_size, verbose=verbose)
    print(mse)
    print(mse2)
    print(mae)
    return model

# Main Functions

In [19]:
def main_cnn(input_df, dur, limit):
    """
    input_df    die Input Matrix (duh)
    dur         Zeitfenster
    limit       Grenze für Train Test Split
    """

    N = input_df.shape[1]
    D = input_df.shape[0]
    train = input_df.iloc[:limit,:]
    test = input_df.iloc[limit-dur:,:]
    x_train,y_train = get_data_subsets(train, dur) 
    x_test,y_test = get_data_subsets(test, dur)
    inp_shape = (x_train.shape[1],N)
    
    """für Dimensions-tests"""
    print('x-train shape: ' + str(x_train.shape))
    print('y-train shape: ' + str(y_train.shape))
    print('x-test shape: ' + str(x_test.shape))
    print('y-test shape: ' + str(y_test.shape))
    print('test shape: ' + str(test.shape))
    
    model = cnn(x_train, y_train, x_test, y_test, inp_shape)
    return model,x_train,y_train,x_test,y_test

In [21]:
def main_fully(input_df, dur, limit):
    """
    input_df    die Input Matrix (duh)
    dur         Zeitfenster
    limit       Grenze für Train Test Split
    """

    N = input_df.shape[1]
    D = input_df.shape[0]
    train = input_df.iloc[:limit,:]
    test = input_df.iloc[limit-dur:,:]
    print("now")
    x_train,y_train = get_data_subsets(train, dur) 
    x_test,y_test = get_data_subsets(test, dur)
    inp_shape = (x_train.shape[1],N)
    
    """für Dimensions-tests"""
    print('x-train shape: ' + str(x_train.shape))
    print('y-train shape: ' + str(y_train.shape))
    print('x-test shape: ' + str(x_test.shape))
    print('y-test shape: ' + str(y_test.shape))
    print('test shape: ' + str(test.shape))
    
    model = fully(x_train, y_train, x_test, y_test, inp_shape)
    return model,x_train,y_train,x_test,y_test

In [22]:
def main_rnn(input_df, dur, limit):
    """
    input_df    die Input Matrix (duh)
    dur         Zeitfenster
    limit       Grenze für Train Test Split
    """

    N = input_df.shape[1]
    D = input_df.shape[0]
    train = input_df.iloc[:limit,:]
    test = input_df.iloc[limit-dur:,:]
    x_train,y_train = get_data_subsets(train, dur) 
    x_test,y_test = get_data_subsets(test, dur)
    inp_shape = (x_train.shape[1],N)
    
    """für Dimensions-tests"""
    print('x-train shape: ' + str(x_train.shape))
    print('y-train shape: ' + str(y_train.shape))
    print('x-test shape: ' + str(x_test.shape))
    print('y-test shape: ' + str(y_test.shape))
    print('test shape: ' + str(test.shape))
    
    model = rnn(x_train, y_train, x_test, y_test, inp_shape)
    return model,x_train,y_train,x_test,y_test

In [ ]:
def main_lstm(input_df, dur, limit):
    """
    input_df    die Input Matrix (duh)
    dur         Zeitfenster
    limit       Grenze für Train Test Split
    """

    N = input_df.shape[1]
    D = input_df.shape[0]
    train = input_df.iloc[:limit,:]
    test = input_df.iloc[limit-dur:,:]
    x_train,y_train = get_data_subsets(train, dur) 
    x_test,y_test = get_data_subsets(test, dur)
    inp_shape = (x_train.shape[1],N)
    
    """für Dimensions-tests"""
    print('x-train shape: ' + str(x_train.shape))
    print('y-train shape: ' + str(y_train.shape))
    print('x-test shape: ' + str(x_test.shape))
    print('y-test shape: ' + str(y_test.shape))
    print('test shape: ' + str(test.shape))
    
    model = lstm_rnn(x_train, y_train, x_test, y_test, inp_shape)
    return model,x_train,y_train,x_test,y_test

# Calculations

In [20]:
model_cnn,x_train,y_train,x_test,y_test = main_cnn(returns, 100, 2000)

1900
518
x-train shape: (1900, 109, 20)
y-train shape: (1900, 20)
x-test shape: (518, 109, 20)
y-test shape: (518, 20)
test shape: (618, 20)
Epoch 1/40
1900/1900 [==============================] - 2s 1ms/step - loss: 5.4347e-04 - mse: 5.4347e-04 - mae: 0.0140
Epoch 2/40
1900/1900 [==============================] - 2s 899us/step - loss: 4.1261e-04 - mse: 4.1261e-04 - mae: 0.0120
Epoch 3/40
1900/1900 [==============================] - 2s 920us/step - loss: 4.1285e-04 - mse: 4.1285e-04 - mae: 0.0120
Epoch 4/40
1900/1900 [==============================] - 2s 879us/step - loss: 4.1273e-04 - mse: 4.1273e-04 - mae: 0.0119
Epoch 5/40
1900/1900 [==============================] - 2s 867us/step - loss: 4.1291e-04 - mse: 4.1291e-04 - mae: 0.0120
Epoch 6/40
1900/1900 [==============================] - 2s 970us/step - loss: 4.1358e-04 - mse: 4.1358e-04 - mae: 0.0120
Epoch 7/40
1900/1900 [==============================] - 2s 915us/step - loss: 4.1317e-04 - mse: 4.1317e-04 - mae: 0.0120
Epoch 8/40
190

In [ ]:
model_fully,x_train,y_train,x_test,y_test = main_fully_connected(returns, 100, 2000)

In [ ]:
model_rnn,x_train,y_train,x_test,y_test = main_rnn(returns, 100, 2000)

1900
518
x-train shape: (1900, 109, 20)
y-train shape: (1900, 20)
x-test shape: (518, 109, 20)
y-test shape: (518, 20)
test shape: (618, 20)
Epoch 1/40
1900/1900 [==============================] - 16s 8ms/step - loss: 4.1667e-04 - mse: 4.1667e-04 - mae: 0.0122
Epoch 2/40
1900/1900 [==============================] - 15s 8ms/step - loss: 4.1305e-04 - mse: 4.1305e-04 - mae: 0.0121
Epoch 3/40
1900/1900 [==============================] - 15s 8ms/step - loss: 4.1247e-04 - mse: 4.1247e-04 - mae: 0.0121
Epoch 4/40
1900/1900 [==============================] - 15s 8ms/step - loss: 4.1261e-04 - mse: 4.1261e-04 - mae: 0.0121
Epoch 5/40
1900/1900 [==============================] - 15s 8ms/step - loss: 4.1293e-04 - mse: 4.1293e-04 - mae: 0.0122
Epoch 6/40
1900/1900 [==============================] - 15s 8ms/step - loss: 4.1242e-04 - mse: 4.1242e-04 - mae: 0.0121
Epoch 7/40
1900/1900 [==============================] - 15s 8ms/step - loss: 4.1034e-04 - mse: 4.1034e-04 - mae: 0.0121
Epoch 8/40
1900/190

In [ ]:
model_lstm,x_train,y_train,x_test,y_test = main_lstm(returns, 100, 2000)

# Summaries

In [ ]:
print(model_fully.summary())

In [ ]:
print(model_cnn.summary())

In [ ]:
print(model_rnn.summary())

In [ ]:
print(model_lstm.summary())

## Tests 

Function to check wether stock value changes were recognised to increase/decrease correctly

In [ ]:
def directional_goodness(model,x_new,y_new):
    """
    directional goodnes gives the factor of correctly predicted signs of first order derivative of returns to false ones
    oder auch: 
    gibt die Anzahl der Beobachtungen an, deren Vorhersage das richtige Vorzeichen hatte (Kurs steigt, Kurs fällt)
    """
    y_pred = model.predict(x_new)
    count = 0
    #print(y_new.shape)
    for i in range(len(y_pred)):
        for j in range(len(y_pred[i])):
            p= y_pred[i,j] > 0
            n = y_new[i,j] > 0
            if n==p:
                count +=1
    print('percentage of correctly predicted directions of returns: ' + str(count/len(y_pred)/len(y_pred[0])))

In [ ]:
directional_goodness(model_cnn,x_test,y_test)

In [ ]:
directional_goodness(model_cnn,x_train,y_train)

In [ ]:
directional_goodness(model_fully,x_test,y_test)

In [ ]:
directional_goodness(model_fully,x_train,y_train)

In [ ]:
directional_goodness(model_rnn,x_test,y_test)

In [ ]:
directional_goodness(model_rnn,x_train,y_train)

Function to check predicted values vs actual measured values used for training

In [ ]:
def internal_goodness(model, x_new, y_new):
    print('compare true to predicted values of internal validation on learned dataset: ')
    y_pred = model.predict(x_new)
    #model.predict nimmt x-werte und gibt die predicteten y-werte zurück
    plt.scatter(y_new, y_pred)

In [ ]:
internal_goodness(model_cnn,x_test,y_test)

In [ ]:
internal_goodness(model_cnn,x_train,y_train)

In [ ]:
internal_goodness(model_fully,x_test,y_test)

In [ ]:
internal_goodness(model_fully,x_train,y_train)

In [ ]:
internal_goodness(model_rnn,x_test,y_test)

In [ ]:
internal_goodness(model_rnn,x_train,y_train)

Function to check predicted values for future data vs actual future data

In [ ]:
def external_goodness(model, x_new, y_new):
    print('compare true to predicted values of external validation set: ')
    y_pred = model.predict(x_new)
    color =  ["r", "b", "g"]
    for i in range(3):
        plt.plot(y_new[:100,i],c=color[i], label = 'true')
        plt.plot(y_pred[:100,i], c=color[i], linestyle='--', label = 'pred')
    plt.legend()

In [ ]:
external_goodness(model_cnn,x_test,y_test)

In [ ]:
external_goodness(model_cnn,x_train,y_train)

In [ ]:
external_goodness(model_fully,x_test,y_test)

In [ ]:
external_goodness(model_fully,x_train,y_train)

In [ ]:
external_goodness(model_rnn,x_test,y_test)

In [ ]:
external_goodness(model_rnn,x_train,y_train)

Test Correlation of all vourses having same values

In [ ]:
x_test_self = x_test.copy()
for i in range(1,20):
    x_test_self[:,:,i] = x_test[:,:,0]
external_goodness(model_cnn, x_test_self, y_test)

In [ ]:
for i in range(1,20):
    x_test2[:,:,i] = x_test[:,:,0]

In [ ]:
external_goodness(model_cnn, x_test2,y_test)

In [ ]:
def largest(array, amount):
    return array.argsort()[:,-amount:]


def compare_top(model, x_test, y_test):
    amount = 5
    mean_all = np.mean(y_test)
    y_pred = model.predict(x_test)
    top_index = largest(y_pred,amount)
    mean_mach = np.mean(y_test[top_index])
    return mean_mach-mean_all
compare_top(model, x_test, y_test)